# 📘 One-to-Many Relationship (Author → Book)

In this example, one `Author` can have many `Book` instances.

- Each `Book` belongs to one `Author`
- This is a **ForeignKey (many-to-one)** relationship

## ✅ Step 1: Define Models

In [ ]:
# models.py
from django.db import models

class Author(models.Model):
    name = models.CharField(max_length=100)

    def __str__(self):
        return self.name

class Book(models.Model):
    title = models.CharField(max_length=200)
    author = models.ForeignKey(Author, related_name='books', on_delete=models.CASCADE)

    def __str__(self):
        return self.title

## ✅ Step 2A: Nested Serializer (Read-Only)

In [ ]:
# serializers.py
from rest_framework import serializers
from .models import Author, Book

class BookSerializer(serializers.ModelSerializer):
    class Meta:
        model = Book
        fields = ['id', 'title']

class AuthorSerializer(serializers.ModelSerializer):
    books = BookSerializer(many=True, read_only=True)

    class Meta:
        model = Author
        fields = ['id', 'name', 'books']

**Example Output**:
```json
{
  "id": 1,
  "name": "J. K. Rowling",
  "books": [
    { "id": 1, "title": "Harry Potter 1" },
    { "id": 2, "title": "Harry Potter 2" }
  ]
}
```

## ✅ Step 2B: PrimaryKeyRelatedField

In [ ]:
# serializers.py
class AuthorSerializer(serializers.ModelSerializer):
    books = serializers.PrimaryKeyRelatedField(many=True, read_only=True)

    class Meta:
        model = Author
        fields = ['id', 'name', 'books']

**Example Output**:
```json
{
  "id": 1,
  "name": "J. K. Rowling",
  "books": [1, 2]
}
```

## ✅ Step 3A: BookSerializer with Writable Author (by ID)

In [ ]:
# serializers.py
class BookSerializer(serializers.ModelSerializer):
    author = serializers.PrimaryKeyRelatedField(queryset=Author.objects.all())

    class Meta:
        model = Book
        fields = ['id', 'title', 'author']

**Input**:
```json
{ "title": "New Book", "author": 1 }
```

## ✅ Step 3B: SlugRelatedField for Author by Name

In [ ]:
# serializers.py
class BookSerializer(serializers.ModelSerializer):
    author = serializers.SlugRelatedField(slug_field='name', queryset=Author.objects.all())

    class Meta:
        model = Book
        fields = ['id', 'title', 'author']

**Input**:
```json
{ "title": "Another Book", "author": "John" }
```

## ✅ Step 3C: StringRelatedField (Read-Only Display)

In [ ]:
# serializers.py
class BookSerializer(serializers.ModelSerializer):
    author = serializers.StringRelatedField()

    class Meta:
        model = Book
        fields = ['id', 'title', 'author']

**Output**:
```json
{ "id": 1, "title": "Book Title", "author": "John" }
```

## 🔁 Summary Table

| Field Type            | Input Example            | Output Example           | Writable | Notes |
|------------------------|---------------------------|----------------------------|----------|-------|
| PrimaryKeyRelatedField | `"author": 1`             | `"author": 1`             | ✅ Yes    | Uses ID |
| SlugRelatedField       | `"author": "John"`       | `"author": "John"`       | ✅ Yes    | Needs unique field |
| StringRelatedField     | N/A                       | `"author": "John"`       | ❌ No     | Uses `__str__()` |
| Nested Serializer      | N/A                       | `{ "books": [..] }`      | ❌ No     | Good for read APIs |